In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive

/content/drive/My Drive


# Import required libraries


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')
from tqdm import tqdm
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

# Helper functions for preprocessing


Preprocessing steps for Profile data set

In [0]:
def profile_convert_date_to_days(profile_df):
  
  '''This function creates new feature days_as_member by calculating 
    number of days since a customer is member using became member column '''
  
    profile_df['became_member_on'] = profile_df['became_member_on'].astype(str).astype('datetime64[ns]', format = "%Y%m%d")
    max_date = profile_df['became_member_on'].max()
    profile_df['days_as_member'] = profile_df['became_member_on'].apply(lambda x: (max_date - x).days)
    profile_df.drop('became_member_on', axis=1, inplace=True)
    return profile_df

def profile_create_buckets(profile_df):
   
    ''' Creates bins for  income and age'''
    
    income_bins = [0,20000,30000,40000,50000,60000,70000,80000,100000,150000,500000]
    profile_df['income_bins'] = pd.cut(profile_df['income'],income_bins, labels = ['0-20K','20K-30K', '30K-40K','40K-50K','50K-60K','60K-70k', '70K-80K','80K-100K', '100K-150K', '150K-500K' ],include_lowest=True)

    age_bins = [0,10,20,30,40,50,60,70,80,90,100,118]
    profile_df['age_bins'] = pd.cut(profile_df['age'],age_bins, labels = ['0-10','10-20','20-30','30-40','40-50', '50-60','60-70', '70-80', '80-90', '90-100', 'Unknown' ],include_lowest=True)
    return profile_df


def profile_preprocessing(profile_df):
  
  ''' Preprocessing steps for profile dataset
       1.Age ==118 is marked as NaN
       2.became_member_on is converted to number_of_days
       3. Bins for Age and Income are created
  '''
    
    profile_df['age'] = profile_df['age'].apply(lambda x: x if x<118 else np.NaN )
    profile_df = profile_convert_date_to_days(profile_df)
    profile_df = profile_create_buckets(profile_df)
    return profile_df
    
    
    
    

Preprocessing steps for Portfolio

In [0]:
def one_hot_channels(portfolio_df):
 
    #one hot encoded channels columns
  
    channels = ['web', 'email', 'mobile', 'social']

    for channel in channels:
        portfolio_df[channel] = portfolio_df.channels.apply(lambda x: channel in x)*1

    #drop channels column
    portfolio_df = portfolio_df.drop('channels', axis=1)
    
    return portfolio_df

def portfolio_preprocessing(portfolio_df):
  
  '''Preprocessing of portfolio datasets include
   1. one hot encoding of channel into individual channels
   2. Add two new features 
      relative difficulty which captures relationship betweeen 
      durationa and difficulty
      reward to difficulty which give reward to difficulty ratio
   
   '''
   
    # one_hot_channels
    portfolio_df = one_hot_channels(portfolio_df)
    
    # Add new features 
    portfolio_df['duration_hours'] = portfolio_df['duration']*24
    portfolio_df['relative_difficulty'] = portfolio_df['difficulty']/portfolio_df['duration']
    portfolio_df['reward_to_difficulty'] = portfolio_df['reward']/portfolio_df['difficulty']
    
    return portfolio_df

Preprocessing of Transcript data set

In [0]:
def transcript_preprocessing(transcript_df):
  
  ''' This function splits the value column into 
  offer_id , amount and corresponding actual value '''
  
    values_df = pd.DataFrame(transcript_df.value.tolist())
    values_df.offer_id.update(values_df['offer id'])
    values_df = values_df.drop('offer id', axis=1)
    return transcript_df.join(values_df).drop('value', axis=1)
  
def transcript_remove_outliers(transcript_df):
  
  ''' This function imputes outlier transaction amounts with transaction 
    amount that got z value just over 2.95 '''
  
  z = np.abs(stats.zscore(transcript_df[transcript_df.event == 'transaction'].amount))
  min_outlier = transcript_df[transcript_df.event == 'transaction'].iloc[np.where(z >2.95)]['amount'].min()
  transcript_df['amount'].values[transcript_df['amount'] > min_outlier] = min_outlier
  return transcript_df
  

# Build functions to prepare full dataset

In [0]:
def merge_all(profile, portfolio, transcript):
  
  """
  Merges Profile, Portfolio and transcript datasets into one single dataset
    
  """
  
    merged_df = transcript.merge(profile, left_on='person', right_on='id').drop('id', axis=1)
    merged_df = merged_df.merge(
        portfolio.rename(columns={'reward': 'reward_t'}),
        left_on='offer_id', right_on='id', how ='left').drop('id', axis=1)
    return merged_df

In [0]:
def split_all(merged_df):
  
  """
  Split the merged data into three datasets based on event
  
  Args:
     merged_df : This merged dataset for one client only
  
  returns:
     received (pd.DataFrame), viewed (pd.DataFrame) and transactions
     
     all above 3 data frames contain events related to single client
  
  """
    received = merged_df[merged_df.event == 'offer received']
    viewed =  merged_df[merged_df.event == 'offer viewed']
    completed =  merged_df[merged_df.event == 'offer completed']
    transactions = merged_df[merged_df.event == 'transaction']
    
    return received,viewed,completed,transactions

In [0]:
def fill_completion(received, completed):
    """
    Looks in the records of one person and checks which offers where completed.
    A 'completed' column is set to 1 when the offer was completed. The finish
    time is also added.
    Args:
        received(pd.DataFrame): As returned from split_transcript
        completed(pd.DataFrame): As returned from split_transcript
    Returns:
        pd.DataFrame: The received dataframe with some new columns.
    """
    results = list()
    for idx, row in received.iterrows():
        record = dict()

        # Identify the record
        record['time'] = row.time
        record['offer_id'] = row.offer_id
        #record['person'] = row.person

        record['valid_till'] = row.time + row.duration_hours
        completion = completed[(completed.offer_id == row.offer_id) &
                               (completed.time >= row.time) &
                               (completed.time <= record['valid_till'])]
        if completion.shape[0] > 0:
            record['completed'] = 1
            record['completed_time'] = completion.time.iloc[0]
        else:
            record['completed'] = 0
            record['completed_time'] = record['valid_till']

        results.append(record)
    results_df = pd.DataFrame(results)
    
    #return results_df
    return received.merge(results_df, on = ['time', 'offer_id'], how = 'left' ) 


In [0]:
def fill_viewed(data, viewed):
    """
    Checks if the offer was viewed in the active period of the offers.
    Also fills following columns  
        'success' that tracks whether an offer completion happened 
         after a view.
         'completed_duration' that tracks how long it took to complete the 
         offer for viewed offer
         'participated_duration' that tracks how long customer has participated
    Args:
        data(pd.DataFrame): As returned from fill_completed
        viewed(pd.DataFrame): As returned from split_transcript
    Returns:
        pd.DataFrame: The received dataframe with some new columns.
    """
    results = list()
    for idx, row in data.iterrows():
        record = dict()

        # Identify the record
        record['time'] = row.time
        record['offer_id'] = row.offer_id

        views = viewed[(viewed.offer_id == row.offer_id) &
                       (viewed.time >= row.time) &
                       (viewed.time <= row.completed_time)]
        if views.shape[0] > 0:
            record['viewed'] = 1
            record['view_time'] = views.time.iloc[0]
            if (record['view_time'] <= row.completed_time) and row.completed:
                record['success'] = 1
                record['completed_duration'] = 1 if row.completed_time == record['view_time'] else \
                                                                         row.completed_time-record['view_time']
            else:
                record['success'] = 0
                record['completed_duration'] = np.nan
                    
            if row.completed:
                record['participated_duration'] = record['completed_duration']
            else:
                record['participated_duration'] = row.valid_till- record['view_time']
        else:
            record['viewed'] = 0
            record['view_time'] = np.nan
            record['success'] = 0
            record['completed_duration'] = np.nan
            record['participated_duration'] = 0

        results.append(record)
    
    results_df = pd.DataFrame(results)

    return data.merge(results_df, on = ['time', 'offer_id'], how = 'left' ) 

In [0]:
def get_non_promotions(data, transactions):
  
  """
    Gets the number of transactions and total amount spent between the promotions
  
    Args:
        data(pd.DataFrame): As returned from fill_viewed
        transactions(pd.DataFrame): As returned from split_transcript
    Returns:
        non_prmo_amount : Total amount by a customer between promotions
        non_promo_tranx : Total number of transactions between promotions
  
  
  """
  
    ranges = list(data[['time', 'valid_till']].itertuples(index=False, name =None))
    #intialize the variables
    tr_in_offer_period =[]
    non_promo_amount =0
    non_promo_tranx =0
    
    for idx, tr_row in transactions.iterrows():
        if any(lower <= tr_row.time <= upper for (lower, upper) in ranges):
            tr_in_offer_period.append(tr_row.time)
        
        if tr_in_offer_period.count(tr_row.time) ==0:
            non_promo_amount =+tr_row.amount
            non_promo_tranx =+1
            
    return non_promo_amount, non_promo_tranx

In [0]:
def fill_spending(data, transactions):
    """
    Fills "spending" and number of transactions related columns.
    The "spending" columns track the transaction amounts of the client in the various
    stages of an offer like "before viewing", "between view to complete" and "after 
    completing the offer". 
  
    The paid reward is also recorded in the column "actual reward" (it is zero if the
    offer was not completed).
    Args:
        data(pd.DataFrame): As returned from fill_viewed
        transactions(pd.DataFrame): As returned from split_transcript
    Returns:
        pd.DataFrame: The received dataframe with some new columns.
    """
    
    #Get total amount spent and number of transactions for client between the offers
    
    total_non_promo_amount, total_non_promo_tranx =get_non_promotions(data, transactions)
    
    results = list()
    
    for idx, row in data.iterrows():
        record = dict()

        # Identify the record
        record['time'] = row.time
        record['offer_id'] = row.offer_id
        
        until_viewed_tr =transactions[(transactions.time >= row.time) &
                                         (transactions.time < row.view_time)]

        until_complete_tr = transactions[(transactions.time >= row.time) &
                                         (transactions.time <= row.completed_time)]
        duration_tr = transactions[(transactions.time >= row.time) &
                                   (transactions.time <= row.valid_till)]
        
        record['spent_before_view'] = until_viewed_tr.amount.sum() if row.viewed == 1 else 0
        record['tranx_before_view'] = until_viewed_tr.amount.count() if row.viewed == 1 else 0
        record['spent_until_complete'] = until_complete_tr.amount.sum() if row.viewed == 1 else 0
        record['tranx_until_complete'] = until_complete_tr.amount.count() if row.viewed == 1 else 0
        record['spent_in_duration'] = duration_tr.amount.sum() if row.viewed == 1 else 0
        record['tranx_in_duration'] = duration_tr.amount.count() if row.viewed == 1 else 0
        record['non_participation_spent'] = duration_tr.amount.sum() if row.viewed == 0 else 0
        record['non_participation_tranx'] = duration_tr.amount.count() if row.viewed == 0 else 0
        record['spent_from_view_to_complete'] = record['spent_until_complete'] - \
                                                record['spent_before_view'] 
        record['tranx_from_view_to_complete'] = record['tranx_until_complete'] - \
                                                record['tranx_before_view']
        record['spent_after_complete'] = record['spent_in_duration'] - \
                                         record['spent_until_complete']
        record['tranx_after_complete'] = record['tranx_in_duration'] - \
                                         record['tranx_until_complete'] 
        record['np_spent_in_duration'] =  record['non_participation_spent'] if row.viewed == 0 else \
                                                record['spent_before_view'] + record['spent_after_complete']
        record['np_tranx_in_duration'] =  record['non_participation_tranx'] if row.viewed == 0 else \
                                                record['tranx_before_view'] + record['tranx_after_complete']
        record['total_spent'] = transactions.amount.sum()
        record['total_tranx'] = transactions.amount.count()
        record['actual_reward'] = row.reward_t if row.completed == 1 else 0
        
        record['total_spent_between_promo'] = total_non_promo_amount
        record['total_tranx_between_promo'] = total_non_promo_tranx
       
        results.append(record)
    
    results_df = pd.DataFrame(results)

    return data.merge(results_df, on = ['time', 'offer_id'], how = 'left' ) 
    

In [0]:
def generate_complete_set(merged_df):
  
  """
  Generate complete set for one client at a time
  """
    received,viewed,completed,transactions = split_all(merged_df)
    data = fill_completion(received, completed)
    data = fill_viewed(data, viewed)
    data = fill_spending(data, transactions)
    return data
 

# Feature generator functions

In [0]:
def get_per_hour_rate(df):
  
  """ 
   Calculates rate of spend and rate of transactions with relative to duration
  """
    
    df['per_hour_spend_in_promo'] = 0
    df['per_hour_tranx_in_promo'] = 0

    df['per_hour_spend_in_promo'] = df[(df.viewed ==1) ]['spent_from_view_to_complete']/df[(df.viewed ==1)]['participated_duration'] 
    df['per_hour_tranx_in_promo'] = df[(df.viewed ==1) ]['tranx_from_view_to_complete']/df[(df.viewed ==1)]['participated_duration']
    
    df['per_hour_spend_in_np'] = df['np_spent_in_duration']/(df['duration_hours']- df['participated_duration'])
    df['per_hour_tranx_in_np'] = df['np_tranx_in_duration']/(df['duration_hours']- df['participated_duration'])
    
    return df

In [0]:
def get_percentages(df):
  
  """
  Calculates rate of completion for successful offers, percentage of spending 
  and percentage of transactions during promotion active period
  
  """
    
    df['rate_of_completion'] = (df[df.success ==1]['duration_hours']- \
                                df[df.success ==1]['participated_duration'])/df[df.success ==1]['duration_hours']
    df['perc_spend_in_promo'] =  df['spent_from_view_to_complete']/ \
                                    (df['spent_from_view_to_complete']+df['np_spent_in_duration'])
    df['perc_tranx_in_promo'] =  df['tranx_from_view_to_complete']/ \
                                    (df['tranx_from_view_to_complete']+df['np_tranx_in_duration'])
    
    return df

In [0]:
def get_spend_per_transaction(df):
  
  """
  Calculates spend per transaction in promotion active period, promotion 
  non-participation duration (i.e before view and after completing)
  
  Also calculates during full duration of an offer.
  
  """
    
    df['spend_per_tranx_in_promo'] = df['spent_from_view_to_complete']/df['tranx_from_view_to_complete']
    df['spend_per_tranx_in_np'] = df['np_spent_in_duration']/ df['np_tranx_in_duration']
    df['spend_per_tranx_in_duration'] = (df['spent_from_view_to_complete']+df['np_spent_in_duration'])/ \
                                          (df['tranx_from_view_to_complete'] + df['np_tranx_in_duration'])

    return df
    

In [0]:
def get_invalid_completes(df):
  
  """
  Assign invalid rewards for completed offers that are not viewed
  
  """
    df['recieved'] = 1
    df[['invalid_complete', 'invalid_reward']] = df[df.viewed == 0][['completed', 'actual_reward']]
    df['completed_status'] = df['completed'].apply(lambda x: 'completed' if x==1 else 'incompleted')
    df['viewed_status'] = df['viewed'].apply(lambda x: 'viewed' if x==1 else 'unviewed')
    return df

In [0]:
def derive_features(df):
  
  """
  Wrapper function to derive features for completed dataset
  
  """
    
    df = get_spend_per_transaction(df)
    df = get_percentages(df)
    df = get_per_hour_rate(df)
    df = get_invalid_completes(df)
    
    return df

# Data Processing steps

In [0]:
profile = pd.read_json('data/profile.json', orient='records', lines=True)
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
transcript= pd.read_json('data/transcript.json', orient='records', lines=True)

In [0]:
profile = profile_preprocessing(profile)
portfolio = portfolio_preprocessing(portfolio)
transcript = transcript_preprocessing(transcript)
transcript = transcript_remove_outliers(transcript)

In [0]:
merged = merge_all(profile, portfolio, transcript)

#will consider only the clients who recieved atleast an offer
profiles_to_consider = pd.DataFrame(merged[merged.event == 'offer received'].groupby('person').event.count())

print('Number of clients who didnt recieve atleast an offer:', profile.shape[0] - profiles_to_consider[0])

(306534, 23)


In [0]:
df = pd.DataFrame() #initialize the dataframe
for profile_id in tqdm(profiles_to_consider.index):
        # take a subset_df for profile_id person
        subset_df = generate_complete_set(merged[merged.person == profile_id])
        df = df.append(subset_df, ignore_index=True)

        

100%|██████████| 16994/16994 [55:44<00:00,  4.01it/s]


In [0]:
print ('shape before ', df.shape)
df = derive_features(df)
print ('shape after ', df.shape)
    

shape before  (76277, 50)
shape after  (76277, 65)


In [0]:
#replace nan, inf with 0

new_cols = ['spend_per_tranx_in_promo',
       'spend_per_tranx_in_np', 'spend_per_tranx_in_duration',
       'rate_of_completion', 'perc_spend_in_promo', 'perc_tranx_in_promo',
       'per_hour_spend_in_promo', 'per_hour_tranx_in_promo',
       'per_hour_spend_in_np', 'per_hour_tranx_in_np','invalid_complete', 'invalid_reward']
df[new_cols] = df[new_cols].replace([np.inf, -np.inf, np.nan], 0)

In [0]:
df.to_csv("data/sbucks_expanded.csv", index=None)

In [0]:
df_exp = df

# Explore complete dataset for insights


In [0]:
df_exp['portfolio_id'] = df_exp['offer_type']+'-'+df_exp['difficulty'].astype(str)+'-'+ df_exp['duration'].astype(str)+'-'+df_exp['reward_t'].astype(str)

In [0]:
customer_encoder = LabelEncoder()

df_exp['customer_id'] = customer_encoder.fit_transform(df_exp['person'])

In [0]:
portfolio_exp = df_exp[['portfolio_id', 'email', 'web', 'social', 'mobile','recieved', 'viewed', 'completed', 'success', 'invalid_complete']].groupby(['portfolio_id']).sum()

In [0]:
portfolio_exp['email_view_rate'] = portfolio_exp['viewed']/portfolio_exp['email']
portfolio_exp['web_view_rate'] = portfolio_exp['viewed']/portfolio_exp['web']
portfolio_exp['social_view_rate'] = portfolio_exp['viewed']/portfolio_exp['social']
portfolio_exp['mobile_view_rate'] = portfolio_exp['viewed']/portfolio_exp['mobile']

In [0]:
portfolio_exp[['email_view_rate', 'web_view_rate', 'social_view_rate', 'mobile_view_rate']].replace([np.inf], 0)

,email_view_rate,web_view_rate,social_view_rate,mobile_view_rate
portfolio_id,,,,
bogo-10.0-5.0-10.0,0.894245,0.894245,0.894245,0.894245
bogo-10.0-7.0-10.0,0.795900,0.000000,0.795900,0.795900
bogo-5.0-5.0-5.0,0.874917,0.874917,0.874917,0.874917
bogo-5.0-7.0-5.0,0.465156,0.465156,0.000000,0.465156
discount-10.0-10.0-2.0,0.899171,0.899171,0.899171,0.899171
discount-10.0-7.0-2.0,0.469078,0.469078,0.000000,0.469078
discount-20.0-10.0-5.0,0.321857,0.321857,0.000000,0.000000
discount-7.0-7.0-3.0,0.883076,0.883076,0.883076,0.883076
informational-0.0-3.0-0.0,0.807561,0.000000,0.807561,0.807561


In [0]:
df_exp[['portfolio_id', 'duration', 'difficulty','relative_difficulty','recieved', 'viewed', 'completed', 'success', 'invalid_complete']].groupby(['portfolio_id','duration', 'difficulty', 'relative_difficulty']).sum()

,,,,recieved,viewed,completed,success,invalid_complete
portfolio_id,duration,difficulty,relative_difficulty,,,,,
bogo-10.0-5.0-10.0,5.0,10.0,2.000000,7593,6790,3331,2769,562.0
bogo-10.0-7.0-10.0,7.0,10.0,1.428571,7658,6095,3688,2630,1058.0
bogo-5.0-5.0-5.0,5.0,5.0,1.000000,7571,6624,4296,3546,750.0
bogo-5.0-7.0-5.0,7.0,5.0,0.714286,7677,3571,4354,2135,2219.0
discount-10.0-10.0-2.0,10.0,10.0,1.000000,7597,6831,5332,4678,654.0
discount-10.0-7.0-2.0,7.0,10.0,1.428571,7632,3580,4025,2149,1876.0
discount-20.0-10.0-5.0,10.0,20.0,2.000000,7668,2468,3440,1360,2080.0
discount-7.0-7.0-3.0,7.0,7.0,1.000000,7646,6752,5165,4410,755.0
informational-0.0-3.0-0.0,3.0,0.0,0.000000,7618,6152,0,0,0.0


In [0]:
portfolio_avg = df_exp[['portfolio_id','viewed','completed','spent_from_view_to_complete', 'spent_in_duration', 'np_spent_in_duration']].groupby(['portfolio_id']).mean()

In [0]:
portfolio_avg 

,viewed,completed,spent_from_view_to_complete,spent_in_duration,np_spent_in_duration
portfolio_id,,,,,
bogo-10.0-5.0-10.0,0.894245,0.438694,10.425849,18.783910,11.703773
bogo-10.0-7.0-10.0,0.795900,0.481588,10.628515,21.445226,17.899971
bogo-5.0-5.0-5.0,0.874917,0.567428,9.129274,18.050930,12.781520
bogo-5.0-7.0-5.0,0.465156,0.567149,5.301847,12.521081,18.085847
discount-10.0-10.0-2.0,0.899171,0.701856,12.470186,36.351974,30.016564
discount-10.0-7.0-2.0,0.469078,0.527385,5.844696,13.186424,17.958508
discount-20.0-10.0-5.0,0.321857,0.448618,5.722139,12.236493,24.105707
discount-7.0-7.0-3.0,0.883076,0.675517,10.281477,25.215086,20.314580
informational-0.0-3.0-0.0,0.807561,0.000000,9.119734,10.695876,3.194190


# Prepare consolidated data set

In [0]:
#df = pd.read_csv("data/sbucks_expanded.csv")
person_df = pd.DataFrame()

In [0]:
#Group by person id to get demographic data

person_df = df.drop_duplicates(['person'])[['person','age','gender', 'income', 'days_as_member', 'income_bins', 'age_bins',
 'total_tranx', 'total_tranx_between_promo', 'total_spent','total_spent_between_promo']]
person_df.set_index('person', inplace = True)

In [0]:
#Sum up the selected numerical features by Person

person_total = df[['person','recieved','viewed','completed', 'success','invalid_complete','actual_reward','invalid_reward', 'duration_hours', 'participated_duration', 
       'tranx_from_view_to_complete', 'np_tranx_in_duration', 'spent_from_view_to_complete','np_spent_in_duration']] .groupby('person').agg(np.sum)

In [0]:
person_total.columns = ['%s%s' % ('total', '_%s' %b if b else '') for b in person_total.columns]

In [0]:
#Average of selected numerical features by person

person_mean = df[['person', 'participated_duration', 'tranx_from_view_to_complete',
                  'np_tranx_in_duration', 'spent_from_view_to_complete','np_spent_in_duration',
                   'spend_per_tranx_in_promo','spend_per_tranx_in_np','spend_per_tranx_in_duration',
                  'rate_of_completion', 'perc_spend_in_promo', 'perc_tranx_in_promo',
                  'per_hour_spend_in_promo', 'per_hour_tranx_in_promo','per_hour_spend_in_np', 'per_hour_tranx_in_np']] .groupby('person').agg(np.mean)

In [0]:
person_mean.columns = ['%s%s' % ('mean', '_%s' %b) for b in person_mean.columns]

In [0]:
#Join data with person_df
person_df = person_df.join(person_total)
person_df = person_df.join(person_mean)

In [0]:
#Get aggregated view of channels

person_viewed_sum = df[df.offer_type != 'informational'][['person','viewed_status', 'web', 'email','mobile', 'social']] .groupby(['person', 'viewed_status']).agg(np.sum)

In [12]:
person_viewed_sum.head()

web  email  mobile  social
person                           viewed_status                            
0009655768c64bdeb2e877511632db8f unviewed       3.0    3.0     3.0     2.0
00116118485d4dfda04fdbaba9a87b5c viewed         2.0    2.0     2.0     2.0
0011e0d4e6b944f998e987f904e8c1e5 viewed         3.0    3.0     2.0     1.0
0020c2b971eb4e9188eac86d93036a77 unviewed       1.0    2.0     2.0     2.0
                                 viewed         2.0    2.0     2.0     2.0

In [0]:
person_viewed_sum = person_viewed_sum.unstack(level = 'viewed_status')
new_columns = ['%s%s' % ('total_%s'%b, '_%s' % a if b else '') for a, b in person_viewed_sum.columns]
person_viewed_sum.columns = person_viewed_sum.columns.droplevel()
person_viewed_sum.columns = new_columns

In [14]:
person_viewed_sum.head()

,total_unviewed_web,total_viewed_web,total_unviewed_email,total_viewed_email,total_unviewed_mobile,total_viewed_mobile,total_unviewed_social,total_viewed_social
person,,,,,,,,
0009655768c64bdeb2e877511632db8f,3.0,NaN,3.0,NaN,3.0,NaN,2.0,NaN
00116118485d4dfda04fdbaba9a87b5c,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
0011e0d4e6b944f998e987f904e8c1e5,NaN,3.0,NaN,3.0,NaN,2.0,NaN,1.0
0020c2b971eb4e9188eac86d93036a77,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0020ccbbb6d84e358d3414a3ff76cffd,NaN,3.0,NaN,3.0,NaN,3.0,NaN,2.0


In [0]:

# Here we want to find if all viewed offers, what channels were missing
person_viewed_sum['total_viewed_no_social'] = (person_viewed_sum['total_viewed_email'] - person_viewed_sum['total_viewed_social']).replace(np.nan, 0)
person_viewed_sum['total_viewed_no_mobile'] = (person_viewed_sum['total_viewed_email'] - person_viewed_sum['total_viewed_mobile']).replace(np.nan, 0)
person_viewed_sum['total_viewed_no_web'] = (person_viewed_sum['total_viewed_email'] - person_viewed_sum['total_viewed_web']).replace(np.nan, 0)



In [0]:
person_viewed_sum.drop(columns = ['total_viewed_web',  'total_viewed_mobile',
       'total_viewed_social'], axis =1, inplace =True)

In [0]:
person_df = person_df.join(person_viewed_sum)

In [0]:
#Get aggregated average difficulty for completed and not completed offers
person_completed_mean = (df[df.offer_type != 'informational'][['person','completed_status', 'difficulty', 'relative_difficulty']] .groupby(['person', 'completed_status'])).agg(np.mean)

In [0]:
person_completed_mean = person_completed_mean.unstack(level = 'completed_status')
new_columns = ['%s%s' % ('mean_%s'%b, '_%s' % a if b else '') for a, b in person_completed_mean.columns]
person_completed_mean.columns = person_completed_mean.columns.droplevel()
person_completed_mean.columns = new_columns

In [20]:
person_completed_mean.head()

,mean_completed_difficulty,mean_incompleted_difficulty,mean_completed_relative_difficulty,mean_incompleted_relative_difficulty
person,,,,
0009655768c64bdeb2e877511632db8f,8.333333,NaN,1.142857,NaN
00116118485d4dfda04fdbaba9a87b5c,NaN,5.0,NaN,1.000000
0011e0d4e6b944f998e987f904e8c1e5,10.666667,NaN,1.238095,NaN
0020c2b971eb4e9188eac86d93036a77,10.000000,10.0,1.333333,1.428571
0020ccbbb6d84e358d3414a3ff76cffd,5.666667,NaN,0.904762,NaN


In [0]:
person_df = person_df.join(person_completed_mean)

In [22]:
#Quick check of columns added so far
person_df.head()

,age,gender,income,days_as_member,income_bins,age_bins,total_tranx,total_tranx_between_promo,total_spent,total_spent_between_promo,total_recieved,total_viewed,total_completed,total_success,total_invalid_complete,total_actual_reward,total_invalid_reward,total_duration_hours,total_participated_duration,total_tranx_from_view_to_complete,total_np_tranx_in_duration,total_spent_from_view_to_complete,total_np_spent_in_duration,mean_participated_duration,mean_tranx_from_view_to_complete,mean_np_tranx_in_duration,mean_spent_from_view_to_complete,mean_np_spent_in_duration,mean_spend_per_tranx_in_promo,mean_spend_per_tranx_in_np,mean_spend_per_tranx_in_duration,mean_rate_of_completion,mean_perc_spend_in_promo,mean_perc_tranx_in_promo,mean_per_hour_spend_in_promo,mean_per_hour_tranx_in_promo,mean_per_hour_spend_in_np,mean_per_hour_tranx_in_np,total_unviewed_web,total_unviewed_email,total_viewed_email,total_unviewed_mobile,total_unviewed_social,total_viewed_no_social,total_viewed_no_mobile,total_viewed_no_web,mean_completed_difficulty,mean_incompleted_difficulty,mean_completed_relative_difficulty,mean_incompleted_relative_difficulty
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,33.0,M,72000.0,461,70K-80K,30-40,8,0,127.60,0.00,5,2,3,0,3.0,9.0,9.0,696.0,108.0,2,12,30.73,188.75,21.6,0.40,2.40,6.1460,37.750,6.146000,8.957000,15.103000,0.000000,0.400000,0.400000,0.120900,0.007500,0.200906,0.013095,3.0,3.0,NaN,3.0,2.0,0.0,0.0,0.0,8.333333,NaN,1.142857,NaN
00116118485d4dfda04fdbaba9a87b5c,NaN,NaN,NaN,92,NaN,NaN,3,1,4.09,3.19,2,2,0,0,0.0,0.0,0.0,240.0,138.0,0,0,0.00,0.00,69.0,0.00,0.00,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,2.0,NaN,NaN,0.0,0.0,0.0,NaN,5.0,NaN,1.000000
0011e0d4e6b944f998e987f904e8c1e5,40.0,O,57000.0,198,50K-60K,30-40,5,1,79.46,13.49,5,5,3,3,0.0,13.0,0.0,744.0,414.0,3,3,56.03,55.02,82.8,0.60,0.60,11.2060,11.004,11.206000,7.805000,10.496667,0.330000,0.379432,0.366667,0.140277,0.007753,0.107220,0.005787,NaN,NaN,3.0,NaN,NaN,2.0,1.0,0.0,10.666667,NaN,1.238095,NaN
0020c2b971eb4e9188eac86d93036a77,59.0,F,90000.0,874,80K-100K,50-60,8,1,196.86,19.49,5,2,3,2,1.0,14.0,2.0,840.0,126.0,2,7,34.87,182.90,25.2,0.40,1.40,6.9740,36.580,6.974000,15.600000,18.584500,0.225000,0.235859,0.250000,0.125000,0.007143,0.266043,0.010253,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,10.000000,10.0,1.333333,1.428571
0020ccbbb6d84e358d3414a3ff76cffd,24.0,F,60000.0,622,50K-60K,20-30,12,1,154.05,16.27,4,4,3,3,0.0,13.0,0.0,528.0,174.0,7,7,95.37,82.18,43.5,1.75,1.75,23.8425,20.545,12.804375,8.093333,12.208750,0.580357,0.524436,0.500000,0.522762,0.040741,0.199545,0.016579,NaN,NaN,3.0,NaN,NaN,1.0,0.0,0.0,5.666667,NaN,0.904762,NaN


In [0]:
# get sum selected numerical features by person and offer type

person_offer_type_total = df[['person','offer_type','recieved','viewed','completed', 'success','invalid_complete','actual_reward','invalid_reward', 
                              'duration_hours', 'participated_duration', 'tranx_from_view_to_complete', 'np_tranx_in_duration', 
                              'spent_from_view_to_complete','np_spent_in_duration']] .groupby(['person', 'offer_type']).agg(np.sum)

In [0]:
person_offer_type_total = person_offer_type_total.unstack(level = 'offer_type')
new_columns = ['%s%s' % ('total_%s'%b, '_%s' % a if b else '') for a, b in person_offer_type_total.columns]
person_offer_type_total.columns = person_offer_type_total.columns.droplevel()
person_offer_type_total.columns = new_columns

In [25]:
person_offer_type_total[person_offer_type_total.columns[19:]].head()

,total_discount_invalid_reward,total_informational_invalid_reward,total_bogo_duration_hours,total_discount_duration_hours,total_informational_duration_hours,total_bogo_participated_duration,total_discount_participated_duration,total_informational_participated_duration,total_bogo_tranx_from_view_to_complete,total_discount_tranx_from_view_to_complete,total_informational_tranx_from_view_to_complete,total_bogo_np_tranx_in_duration,total_discount_np_tranx_in_duration,total_informational_np_tranx_in_duration,total_bogo_spent_from_view_to_complete,total_discount_spent_from_view_to_complete,total_informational_spent_from_view_to_complete,total_bogo_np_spent_in_duration,total_discount_np_spent_in_duration,total_informational_np_spent_in_duration
person,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,4.0,0.0,120.0,408.0,168.0,0.0,0.0,108.0,0.0,0.0,2.0,2.0,10.0,0.0,0.00,0.00,30.73,22.68,166.07,0.0
00116118485d4dfda04fdbaba9a87b5c,NaN,NaN,240.0,NaN,NaN,138.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.00,NaN,NaN,0.00,NaN,NaN
0011e0d4e6b944f998e987f904e8c1e5,0.0,0.0,168.0,408.0,168.0,60.0,210.0,144.0,1.0,2.0,0.0,2.0,1.0,0.0,22.05,33.98,0.00,31.99,23.03,0.0
0020c2b971eb4e9188eac86d93036a77,2.0,0.0,288.0,480.0,72.0,84.0,42.0,0.0,1.0,1.0,0.0,0.0,5.0,2.0,17.24,17.63,0.00,0.00,131.80,51.1
0020ccbbb6d84e358d3414a3ff76cffd,0.0,0.0,288.0,168.0,72.0,48.0,54.0,72.0,2.0,1.0,4.0,4.0,3.0,0.0,24.85,11.65,58.87,47.24,34.94,0.0


In [0]:
#valid earned rewards

person_offer_type_total['total_bogo_earned_reward'] = person_offer_type_total['total_bogo_actual_reward']- person_offer_type_total['total_bogo_invalid_reward']
person_offer_type_total['total_discount_earned_reward'] = person_offer_type_total['total_discount_actual_reward']- person_offer_type_total['total_discount_invalid_reward']

In [0]:
person_df = person_df.join(person_offer_type_total)

In [0]:
#get average of numerical features by person and offer type

person_offer_type_mean = df[['person','offer_type','tranx_from_view_to_complete', 'spent_from_view_to_complete',
                             'np_tranx_in_duration', 'np_spent_in_duration','spend_per_tranx_in_promo',
                             'spend_per_tranx_in_np','spend_per_tranx_in_duration',
                             'rate_of_completion', 'perc_spend_in_promo', 
                             'perc_tranx_in_promo','per_hour_spend_in_promo', 
                             'per_hour_tranx_in_promo','per_hour_spend_in_np',
                             'per_hour_tranx_in_np']].groupby(['person', 'offer_type']).agg(np.mean)

In [0]:

person_offer_type_mean = person_offer_type_mean.unstack(level = 'offer_type')
new_columns = ['%s%s' % ('mean_%s'%b, '_%s' % a if b else '') for a, b in person_offer_type_mean.columns]
person_offer_type_mean.columns = person_offer_type_mean.columns.droplevel()
person_offer_type_mean.columns = new_columns

In [30]:
person_offer_type_mean.head()

,mean_bogo_tranx_from_view_to_complete,mean_discount_tranx_from_view_to_complete,mean_informational_tranx_from_view_to_complete,mean_bogo_spent_from_view_to_complete,mean_discount_spent_from_view_to_complete,mean_informational_spent_from_view_to_complete,mean_bogo_np_tranx_in_duration,mean_discount_np_tranx_in_duration,mean_informational_np_tranx_in_duration,mean_bogo_np_spent_in_duration,mean_discount_np_spent_in_duration,mean_informational_np_spent_in_duration,mean_bogo_spend_per_tranx_in_promo,mean_discount_spend_per_tranx_in_promo,mean_informational_spend_per_tranx_in_promo,mean_bogo_spend_per_tranx_in_np,mean_discount_spend_per_tranx_in_np,mean_informational_spend_per_tranx_in_np,mean_bogo_spend_per_tranx_in_duration,mean_discount_spend_per_tranx_in_duration,mean_informational_spend_per_tranx_in_duration,mean_bogo_rate_of_completion,mean_discount_rate_of_completion,mean_informational_rate_of_completion,mean_bogo_perc_spend_in_promo,mean_discount_perc_spend_in_promo,mean_informational_perc_spend_in_promo,mean_bogo_perc_tranx_in_promo,mean_discount_perc_tranx_in_promo,mean_informational_perc_tranx_in_promo,mean_bogo_per_hour_spend_in_promo,mean_discount_per_hour_spend_in_promo,mean_informational_per_hour_spend_in_promo,mean_bogo_per_hour_tranx_in_promo,mean_discount_per_hour_tranx_in_promo,mean_informational_per_hour_tranx_in_promo,mean_bogo_per_hour_spend_in_np,mean_discount_per_hour_spend_in_np,mean_informational_per_hour_spend_in_np,mean_bogo_per_hour_tranx_in_np,mean_discount_per_hour_tranx_in_np,mean_informational_per_hour_tranx_in_np
person,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,0.0,0.0,1.0,0.000,0.000,15.365,2.0,5.0,0.0,22.68,83.035,0.0,0.000,0.000,15.3650,11.340000,16.722500,0.00,11.340000,16.72250,15.3650,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000,1.0,0.000000,0.000000,0.302250,0.000000,0.000000,0.018750,0.189000,0.407765,0.000000,0.016667,0.024405,0.000000
00116118485d4dfda04fdbaba9a87b5c,0.0,NaN,NaN,0.000,NaN,NaN,0.0,NaN,NaN,0.00,NaN,NaN,0.000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN
0011e0d4e6b944f998e987f904e8c1e5,1.0,1.0,0.0,22.050,16.990,0.000,2.0,0.5,0.0,31.99,11.515,0.0,22.050,16.990,0.0000,15.995000,11.515000,0.00,18.013333,17.23500,0.0000,0.642857,0.503571,0.0,0.408031,0.744565,0.0,0.333333,0.750,0.0,0.367500,0.166941,0.000000,0.016667,0.011048,0.000000,0.296204,0.119948,0.000000,0.018519,0.005208,0.000000
0020c2b971eb4e9188eac86d93036a77,0.5,0.5,0.0,8.620,8.815,0.000,0.0,2.5,2.0,0.00,65.900,51.1,8.620,8.815,0.0000,0.000000,26.225000,25.55,8.620000,25.06625,25.5500,0.150000,0.412500,0.0,0.500000,0.089647,0.0,0.500000,0.125,0.0,0.102619,0.209881,0.000000,0.005952,0.011905,0.000000,0.000000,0.310246,0.709722,0.000000,0.011742,0.027778
0020ccbbb6d84e358d3414a3ff76cffd,1.0,1.0,4.0,12.425,11.650,58.870,2.0,3.0,0.0,23.62,34.940,0.0,12.425,11.650,14.7175,10.363333,11.646667,0.00,11.235000,11.64750,14.7175,0.821429,0.678571,0.0,0.423844,0.250054,1.0,0.375000,0.250,1.0,0.528833,0.215741,0.817639,0.044444,0.018519,0.055556,0.245844,0.306491,0.000000,0.020000,0.026316,0.000000


In [0]:
person_df = person_df.join(person_offer_type_mean)

In [32]:
# Number of features added so far

person_df.shape

(16994, 133)

In [0]:
def get_offer_conversion(person_df):
    
    """
    Calculates conversion of received offers to viewed, recieved to success,
    recieved to completed and received to invalid rates
    
    """
  
    person_df['overall_completed_rate'] = person_df['total_completed']/person_df['total_recieved']
    person_df['overall_invalid_rate'] = person_df['total_invalid_complete']/person_df['total_recieved']
    person_df['bogo_completed_rate'] = person_df['total_bogo_completed']/person_df['total_bogo_recieved']
    person_df['discount_completed_rate'] = person_df['total_discount_completed']/person_df['total_discount_recieved']
    person_df['bogo_invalid_rate'] = person_df['total_bogo_invalid_complete']/person_df['total_bogo_recieved']
    person_df['discount_invalid_rate'] = person_df['total_discount_invalid_complete']/person_df['total_discount_recieved']
    person_df['informational_completed_rate'] = person_df['total_informational_completed']/person_df['total_informational_recieved']
    person_df['overall_success_rate'] = person_df['total_success']/person_df['total_recieved']
    person_df['bogo_success_rate'] = person_df['total_bogo_success']/person_df['total_bogo_recieved']
    person_df['discount_success_rate'] = person_df['total_discount_success']/person_df['total_discount_recieved']
    person_df['informational_success_rate'] = person_df['total_informational_success']/person_df['total_informational_recieved']
    person_df['overall_viewed_rate'] = person_df['total_viewed']/person_df['total_recieved']
    person_df['bogo_viewed_rate'] = person_df['total_bogo_viewed']/person_df['total_bogo_recieved']
    person_df['discount_viewed_rate'] = person_df['total_discount_viewed']/person_df['total_discount_recieved']
    person_df['informational_viewed_rate'] = person_df['total_informational_viewed']/person_df['total_informational_recieved']
    return person_df
  

In [35]:
person_df = get_offer_conversion(person_df)
person_df.shape

(16994, 148)

In [0]:
person_df.to_csv("data/sbucks_consolidated.csv", index='person')